# Data & Config

In [87]:
import pandas as pd
import pydicom
import numpy as np
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import albumentations as A
import timm

In [89]:
IMG_SIZE = (512, 512)
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE * 2
AUG_PROB = 0.75
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LR = 3e-5

In [36]:
df = pd.read_csv("/kaggle/input/rsna-lsdc-data/train.csv")

df = df[df["series_description"] == "Sagittal T1"]

cols_to_include = ["study_id", "series_id", "instance_number"] + list(filter(lambda x: x.startswith("spinal_canal_stenosis"), df.columns))
df = df[cols_to_include]

df.reset_index(drop=True, inplace=True)
df = pd.get_dummies(df, dtype=int)

NUM_CLASSES = df.shape[1] - 3

df.head()

,study_id,series_id,instance_number,spinal_canal_stenosis_l1_l2_Moderate,spinal_canal_stenosis_l1_l2_Normal/Mild,spinal_canal_stenosis_l1_l2_Severe,spinal_canal_stenosis_l2_l3_Moderate,spinal_canal_stenosis_l2_l3_Normal/Mild,spinal_canal_stenosis_l2_l3_Severe,spinal_canal_stenosis_l3_l4_Moderate,spinal_canal_stenosis_l3_l4_Normal/Mild,spinal_canal_stenosis_l3_l4_Severe,spinal_canal_stenosis_l4_l5_Moderate,spinal_canal_stenosis_l4_l5_Normal/Mild,spinal_canal_stenosis_l4_l5_Severe,spinal_canal_stenosis_l5_s1_Moderate,spinal_canal_stenosis_l5_s1_Normal/Mild,spinal_canal_stenosis_l5_s1_Severe
0,1737682527,1258728011,12,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
1,1737682527,1258728011,18,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
2,1737682527,1258728011,9,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
3,1737682527,1258728011,14,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
4,1737682527,1258728011,11,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0


# Utils

In [37]:
def load_dicom(src_path, resize_shape):

    dicom_data = pydicom.dcmread(src_path).pixel_array
    resized_image = (dicom_data / np.max(dicom_data) * 255).astype(np.uint8)
    resized_image = cv2.resize(resized_image, resize_shape)
    
    return resized_image

# Dataset & DataLoader

In [63]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=.2, random_state=42)
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [64]:
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=AUG_PROB),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=AUG_PROB),

    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
    # A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.CoarseDropout(max_holes=16, max_height=64, max_width=64, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),    
    A.Normalize(mean=0.5, std=0.5)
])

transforms_valid = A.Compose([
    # A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:180: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [65]:
class LSDCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __len__(self): return len(self.df)
    
    def __getitem__(self, i):
        
        row = self.df.loc[i]
        study_id = row["study_id"]
        series_id = row["series_id"]
        instance_number = row["instance_number"]
        
        dcm_src = f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/{study_id}/{series_id}/{instance_number}.dcm"
        image = load_dicom(dcm_src, IMG_SIZE)
        
        if self.transforms is not None:
            image = self.transforms(image=image)["image"]
        image = torch.tensor(np.expand_dims(image, axis=0), dtype=torch.float)
        
        columns = list(self.df.columns)[3:]
        target = torch.tensor(row[columns].values, dtype=torch.float)
        
        return image, target

In [66]:
train_dataset = LSDCDataset(train_df, transforms=transforms_train)
train_dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

valid_dataset = LSDCDataset(valid_df, transforms=transforms_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False)

# Modelling

In [92]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.cnn_model = timm.create_model('efficientnet_b0', num_classes=num_classes, pretrained=True)
    
    def forward(self, X):
        X = self.conv_layer(X)
        X = self.cnn_model(X)
        
        return X

# Training

In [96]:
model = Model(NUM_CLASSES)
model.to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train_epoch():
    pass